In [1]:
# https://keras.io/
!pip install -q keras
import keras

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import imgaug as ia
from imgaug import augmenters as iaa
from imgaug import parameters as iap

import cv2
import numpy as np

from clr_callback import *

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 100

l = 36
num_filter = 12

compression = 0.5
dropout_rate = 0.1

clr_triangular = CyclicLR(base_lr=0.05, max_lr=0.5, step_size=2000., mode='triangular2')

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(len(x_train))

50000


In [6]:
augmented_X, augmented_Y = [], []

noise = iaa.AdditiveGaussianNoise(scale=0.1*255)
translation_mat = np.float32([[1,0,10],[0,1,10]])

for image,label in zip(x_train, y_train):
    augmented_X.append(image)
    augmented_Y.append(label)
    
    augmented_X.append(noise.augment_image(image))
    augmented_Y.append(label)

    augmented_X.append(cv2.GaussianBlur(image,(3,3),3))
    augmented_Y.append(label)
    
    augmented_X.append(cv2.flip(image,0))
    augmented_Y.append(label)
    
    augmented_X.append(cv2.warpAffine(image, translation_mat, (img_width,img_height)))
    augmented_Y.append(label)
    
    crop_img = image[3:29, 3:29]
    augmented_X.append(cv2.resize(crop_img, (img_width,img_height)))
    augmented_Y.append(label)

In [7]:
# Convert Images and Steering values to numpy arrays since keras requires them in that form
train_images = np.array(augmented_X)
train_labels = np.array(augmented_Y)
# print(len(X_train), len(Y_train))

train_images = train_images.astype('float32')
x_test = x_test.astype('float32')

train_images = train_images / 255
x_test = x_test / 255

In [8]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [9]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [10]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [11]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = MaxPooling2D(2)(input)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(First_Conv2D)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 16, 16, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 12)   324         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [13]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [14]:
filepath="train6\\model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [clr_triangular, checkpoint]

In [15]:
model.fit(train_images, train_labels,
          batch_size = batch_size, 
          epochs=epochs,
          callbacks=callbacks_list,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 300000 samples, validate on 10000 samples
Epoch 1/100
300000/300000 [==============================] - 883s 3ms/step - loss: 1.6709 - acc: 0.3763 - val_loss: 1.7968 - val_acc: 0.4057

Epoch 00001: val_acc improved from -inf to 0.40570, saving model to train6\model-01-0.41.hdf5
Epoch 2/100
300000/300000 [==============================] - 854s 3ms/step - loss: 1.2545 - acc: 0.5464 - val_loss: 1.2338 - val_acc: 0.5901

Epoch 00002: val_acc improved from 0.40570 to 0.59010, saving model to train6\model-02-0.59.hdf5
Epoch 3/100
300000/300000 [==============================] - 853s 3ms/step - loss: 1.1609 - acc: 0.5823 - val_loss: 1.0031 - val_acc: 0.6415

Epoch 00003: val_acc improved from 0.59010 to 0.64150, saving model to train6\model-03-0.64.hdf5
Epoch 4/100
300000/300000 [==============================] - 852s 3ms/step - loss: 1.0304 - acc: 0.6313 - val_loss: 0.9969 - val_acc: 0.6547

Epoch 00004: val_acc improved from 0.64150 to 0.65470, saving model to train6\model-04-0.65.h

300000/300000 [==============================] - 818s 3ms/step - loss: 0.5218 - acc: 0.8168 - val_loss: 0.7352 - val_acc: 0.7778

Epoch 00040: val_acc did not improve
Epoch 41/100
300000/300000 [==============================] - 818s 3ms/step - loss: 0.5181 - acc: 0.8184 - val_loss: 0.6906 - val_acc: 0.7881

Epoch 00041: val_acc did not improve
Epoch 42/100
300000/300000 [==============================] - 818s 3ms/step - loss: 0.5145 - acc: 0.8201 - val_loss: 0.7410 - val_acc: 0.7776

Epoch 00042: val_acc did not improve
Epoch 43/100
300000/300000 [==============================] - 819s 3ms/step - loss: 0.5070 - acc: 0.8221 - val_loss: 0.6778 - val_acc: 0.7951

Epoch 00043: val_acc improved from 0.79350 to 0.79510, saving model to train6\model-43-0.80.hdf5
Epoch 44/100
300000/300000 [==============================] - 819s 3ms/step - loss: 0.5035 - acc: 0.8231 - val_loss: 0.7481 - val_acc: 0.7814

Epoch 00044: val_acc did not improve
Epoch 45/100
300000/300000 [=========================

KeyboardInterrupt: 

In [ ]:
# # Test the model
# score = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])